In [1]:
import gc
import os
from pathlib import Path
import random
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,timedelta
import sklearn
import matplotlib.dates as mdates
%matplotlib inline
#from pyecharts.charts import Map,Geo
#from pyecharts import options as opts
#from pyecharts.globals import GeoType,RenderType
import json

import csv
#from plotly import tools, subplots
#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.express as px
#import plotly.io as pio
#pio.templates.default = "plotly_dark"
#import plotly.offline as pltoff
#from plotly.graph_objs import *
#pltoff.init_notebook_mode(connected=True)

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
#import lightgbm as lgb
#import xgboost as xgb
# --- setup ---
#pd.set_option('max_columns', 50)

In [2]:
global_data = pd.read_csv('time_series_covid19_confirmed_US.csv')
global_data

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6092,6117,6121,6143,6172,6203,6228,6248,6264,6270
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,19392,19433,19461,19554,19635,19670,19698,19714,19732,19758
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2067,2070,2074,2084,2095,2099,2106,2113,2115,2116
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,2414,2416,2417,2432,2437,2442,2445,2449,2450,2450
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,6040,6042,6043,6058,6072,6086,6084,6095,6097,6102
5,84001011,US,USA,840,1011.0,Bullock,Alabama,US,32.100305,-85.712655,...,1149,1151,1153,1160,1165,1163,1163,1167,1169,1169
6,84001013,US,USA,840,1013.0,Butler,Alabama,US,31.753001,-86.680575,...,1938,1940,1945,1948,1951,1951,1952,1956,1961,1968
7,84001015,US,USA,840,1015.0,Calhoun,Alabama,US,33.774837,-85.826304,...,12915,12940,13017,13063,13090,13175,13202,13232,13275,13300
8,84001017,US,USA,840,1017.0,Chambers,Alabama,US,32.913601,-85.390727,...,3364,3367,3367,3382,3393,3392,3399,3406,3410,3413
9,84001019,US,USA,840,1019.0,Cherokee,Alabama,US,34.178060,-85.606390,...,1753,1753,1753,1757,1763,1764,1764,1768,1770,1770


In [3]:
global_data.rename({'Province_State': 'province', 'Admin2': 'admin2'}, axis=1, inplace=True)

In [4]:
global_data_v2 = global_data.drop(['Lat', 'Long_', 'UID', 'iso2', 'iso3', 'FIPS', 'Country_Region',  'code3'],axis=1)
global_data_v2

,admin2,province,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21
0,Autauga,Alabama,"Autauga, Alabama, US",0,0,0,0,0,0,0,...,6092,6117,6121,6143,6172,6203,6228,6248,6264,6270
1,Baldwin,Alabama,"Baldwin, Alabama, US",0,0,0,0,0,0,0,...,19392,19433,19461,19554,19635,19670,19698,19714,19732,19758
2,Barbour,Alabama,"Barbour, Alabama, US",0,0,0,0,0,0,0,...,2067,2070,2074,2084,2095,2099,2106,2113,2115,2116
3,Bibb,Alabama,"Bibb, Alabama, US",0,0,0,0,0,0,0,...,2414,2416,2417,2432,2437,2442,2445,2449,2450,2450
4,Blount,Alabama,"Blount, Alabama, US",0,0,0,0,0,0,0,...,6040,6042,6043,6058,6072,6086,6084,6095,6097,6102
5,Bullock,Alabama,"Bullock, Alabama, US",0,0,0,0,0,0,0,...,1149,1151,1153,1160,1165,1163,1163,1167,1169,1169
6,Butler,Alabama,"Butler, Alabama, US",0,0,0,0,0,0,0,...,1938,1940,1945,1948,1951,1951,1952,1956,1961,1968
7,Calhoun,Alabama,"Calhoun, Alabama, US",0,0,0,0,0,0,0,...,12915,12940,13017,13063,13090,13175,13202,13232,13275,13300
8,Chambers,Alabama,"Chambers, Alabama, US",0,0,0,0,0,0,0,...,3364,3367,3367,3382,3393,3392,3399,3406,3410,3413
9,Cherokee,Alabama,"Cherokee, Alabama, US",0,0,0,0,0,0,0,...,1753,1753,1753,1757,1763,1764,1764,1768,1770,1770


In [5]:
whole_n_countries = []
for i in [1, 10, 100, 1000, 5000, 10000, 20000, 30000, 40000, 44000, 45000, 45600, 46000, 50000]:
    n_countries = 0
    for j in range(len(global_data_v2.iloc[:, -1])):
        if global_data_v2.iloc[:, -1][j] > i:
            n_countries +=1
    
    whole_n_countries.append(n_countries)
    
whole_n_countries

[3238, 3233, 3170, 2355, 900, 531, 288, 195, 140, 124, 122, 120, 118, 109]

In [6]:
#得到所需确诊病例的地区的index， according to the whole cases for each area
#note different admin2 probabibly will have the same provice
need_areas=[] 
for k in range(len(global_data_v2.iloc[:, -1])):  
    if global_data_v2.iloc[k, -1] > 20000:
        need_areas.append(global_data_v2.loc[k, 'Combined_Key'])

In [7]:
global_data_v3 = global_data_v2.query('Combined_Key == @need_areas').reset_index()

In [8]:
global_data_v3 = global_data_v3.drop('index',axis=1)
global_data_v3

,admin2,province,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21
0,Jefferson,Alabama,"Jefferson, Alabama, US",0,0,0,0,0,0,0,...,70078,70177,70249,70332,70466,70629,70764,70906,70986,71073
1,Madison,Alabama,"Madison, Alabama, US",0,0,0,0,0,0,0,...,31869,31957,32008,32088,32203,32255,32307,32355,32405,32425
2,Mobile,Alabama,"Mobile, Alabama, US",0,0,0,0,0,0,0,...,35289,35345,35378,35725,35810,35894,35937,36044,36108,36139
3,Montgomery,Alabama,"Montgomery, Alabama, US",0,0,0,0,0,0,0,...,22234,22294,22319,22369,22417,22462,22502,22536,22565,22586
4,Shelby,Alabama,"Shelby, Alabama, US",0,0,0,0,0,0,0,...,21565,21604,21656,21714,21773,21820,21848,21892,21929,21968
5,Tuscaloosa,Alabama,"Tuscaloosa, Alabama, US",0,0,0,0,0,0,0,...,23737,23783,23860,23913,23961,23996,24024,24093,24110,24184
6,Anchorage,Alaska,"Anchorage,Alaska,US",0,0,0,0,0,0,0,...,26359,26359,26448,26466,26524,26565,26607,26607,26607,26733
7,Maricopa,Arizona,"Maricopa, Arizona, US",0,0,0,0,1,1,1,...,503686,504992,506046,506662,507480,507951,508997,509683,510360,511055
8,Mohave,Arizona,"Mohave, Arizona, US",0,0,0,0,0,0,0,...,20812,20868,20931,20961,20992,21050,21110,21137,21183,21218
9,Pima,Arizona,"Pima, Arizona, US",0,0,0,0,0,0,0,...,108094,108341,108479,108630,108772,108952,109137,109320,109488,109601


In [9]:
global_data_v4 = global_data_v3.loc[:, '5/1/20':'3/1/21']  # cases large than 20000
global_data_v4

,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,...,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21
0,941,961,986,1000,1030,1055,1072,1100,1115,1126,...,70078,70177,70249,70332,70466,70629,70764,70906,70986,71073
1,236,244,245,249,249,249,253,256,259,260,...,31869,31957,32008,32088,32203,32255,32307,32355,32405,32425
2,1139,1165,1191,1218,1259,1312,1356,1407,1453,1464,...,35289,35345,35378,35725,35810,35894,35937,36044,36108,36139
3,352,376,408,435,456,475,510,528,567,604,...,22234,22294,22319,22369,22417,22462,22502,22536,22565,22586
4,325,327,332,337,342,346,352,362,366,371,...,21565,21604,21656,21714,21773,21820,21848,21892,21929,21968
5,220,226,228,250,254,260,271,280,284,289,...,23737,23783,23860,23913,23961,23996,24024,24093,24110,24184
6,188,188,190,192,193,193,194,197,198,198,...,26359,26359,26448,26466,26524,26565,26607,26607,26607,26733
7,4156,4421,4585,4751,4929,5138,5196,5525,5779,5827,...,503686,504992,506046,506662,507480,507951,508997,509683,510360,511055
8,122,127,135,141,147,150,158,171,178,193,...,20812,20868,20931,20961,20992,21050,21110,21137,21183,21218
9,1267,1300,1326,1346,1379,1425,1465,1520,1554,1585,...,108094,108341,108479,108630,108772,108952,109137,109320,109488,109601


In [10]:
#得到每天新增病例的序列
daily_add_all=[]
for k in range(len(global_data_v4)):
    daily_add=[]
    S_da_t = global_data_v4.iloc[k].values
    for i in range(1, len(S_da_t)):
        differ = S_da_t[i] - S_da_t[i-1]
        if differ < 0 or differ == 0:
            differ = 0.00001
            
        daily_add.append(differ)
            
    daily_add_all.append(daily_add)

In [11]:
G_t_all=[]
for k in range(len(daily_add_all)):
    S_t = daily_add_all[k]
    S_t_all = list(map(lambda x: np.log(x), S_t))
    differS_all=[]
    for i in range(1, len(S_t_all)):
        differS = S_t_all[i] - S_t_all[i-1]
        if differS == 0 or differS < 0:
            differS = 0.00001
            
        differS_all.append(differS)
            
    G_t_all.append(differS_all)

In [12]:
Cij_all=[]
for i in range(len(G_t_all)):
    Cij_row=[]
    for j in range(len(G_t_all)):
        sdi = np.sqrt(np.mean(list(map(lambda x: pow(x, 2), G_t_all[i]))) - pow(np.mean(G_t_all[i]), 2))
        sdj = np.sqrt(np.mean(list(map(lambda x: pow(x, 2), G_t_all[j]))) - pow(np.mean(G_t_all[j]), 2))
        Cij = (np.mean([a*b for a,b in zip(G_t_all[i], G_t_all[j])]) - np.mean(G_t_all[i]) * np.mean(G_t_all[j])) / (sdi * sdj)
        Cij_row.append(Cij)
        
    Cij_all.append(Cij_row)
    
Cij_all = np.array(Cij_all)

In [13]:
mat = np.matrix(Cij_all)
with open('outfile_dailyAdd_US_L20000.txt','wb') as f:
    for line in mat:
        np.savetxt(f, line, fmt='%.2f')